# ***Feature Engineering***

# 📚 ***Import libraries***

In [1]:
# base libraries for data science
from pathlib import Path

import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

# 💾 ***Load data***

In [2]:
DATA_DIR = Path.cwd().resolve().parents[1] / "data"

nyc_houses_df = pd.read_parquet(
    DATA_DIR / "02_intermediate/nyc_houses_fixed.parquet", engine="pyarrow"
)

In [3]:
# print library version for reproducibility

print("Pandas version: ", pd.__version__)
print("sklearn version: ", sk.__version__)

Pandas version:  2.2.3
sklearn version:  1.6.0


# 👷 ***Data preparation***

In [4]:
nyc_houses_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137276 entries, 0 to 137275
Data columns (total 20 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   BUILDING CLASS AT PRESENT       136043 non-null  category      
 1   COMMERCIAL UNITS                137274 non-null  Int64         
 2   BUILDING CLASS CATEGORY         137274 non-null  category      
 3   BUILDING CLASS AT TIME OF SALE  137274 non-null  category      
 4   LOT                             137274 non-null  float64       
 5   TOTAL UNITS                     137274 non-null  Int64         
 6   TAX CLASS AT TIME OF SALE       137274 non-null  float64       
 7   ADDRESS                         137274 non-null  category      
 8   TAX CLASS AT PRESENT            136043 non-null  category      
 9   ZIP CODE                        137274 non-null  float64       
 10  SALE DATE                       137274 non-null  datetim

***Elimination of duplicates***

This is done to prioritize that the duplicates to be deleted are not values where SALE PRICE has a value.

In [5]:
# Count duplicates before removal
initial_duplicates = nyc_houses_df.duplicated().sum()

# Sort so that non-null 'SALE PRICE' values come first
nyc_houses_df = nyc_houses_df.sort_values(by='SALE PRICE', ascending=False, na_position='last')

# Remove duplicates across all columns, keeping the first occurrence
nyc_houses_df = nyc_houses_df.drop_duplicates(keep='first')

# Count duplicates after cleaning
remaining_duplicates = nyc_houses_df.duplicated().sum()

# Print summary of duplicates removed
print(f"Duplicates removed: {initial_duplicates}")
print(f"Duplicates remaining after cleaning: {remaining_duplicates}")


Duplicates removed: 70126
Duplicates remaining after cleaning: 0


In [6]:
nyc_houses_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 67150 entries, 49566 to 67633
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   BUILDING CLASS AT PRESENT       66543 non-null  category      
 1   COMMERCIAL UNITS                67149 non-null  Int64         
 2   BUILDING CLASS CATEGORY         67149 non-null  category      
 3   BUILDING CLASS AT TIME OF SALE  67149 non-null  category      
 4   LOT                             67149 non-null  float64       
 5   TOTAL UNITS                     67149 non-null  Int64         
 6   TAX CLASS AT TIME OF SALE       67149 non-null  float64       
 7   ADDRESS                         67149 non-null  category      
 8   TAX CLASS AT PRESENT            66543 non-null  category      
 9   ZIP CODE                        67149 non-null  float64       
 10  SALE DATE                       67149 non-null  datetime64[ns]
 11  NEI

***For visualization purposes, we are going to see again how many nulls (and zero values) each column has***

In [7]:
def missing_and_zero_values_summary(df):
    """
    Generate a summary of missing and zero values in a DataFrame.

    Parameters:
        df (pd.DataFrame): The DataFrame to analyze.

    Returns:
        pd.DataFrame: A summary DataFrame showing the count and percentage of null and zero values per column.
    """
    nulos_count = df.isnull().sum()
    nulos_percentage = (nulos_count / len(df)) * 100
    zero_values_count = df.eq(0).sum()
    zero_values_percentage = (zero_values_count / len(df)) * 100

    summary_df = pd.DataFrame({
        'Nulls': nulos_count,
        'Percentage Nulls (%)': nulos_percentage,
        'Zero values': zero_values_count,
        'Percentage Zeros (%)': zero_values_percentage
    }).sort_values(by='Percentage Nulls (%)', ascending=False)

    return summary_df


In [8]:
missing_and_zero_values_summary(nyc_houses_df)

,Nulls,Percentage Nulls (%),Zero values,Percentage Zeros (%)
APARTMENT NUMBER,51991,77.425168,0,0.0
GROSS SQUARE FEET,21968,32.714818,9134,13.602383
LAND SQUARE FEET,20868,31.076694,8276,12.324646
SALE PRICE,11425,17.014147,8009,11.927029
TAX CLASS AT PRESENT,607,0.903946,0,0.0
BUILDING CLASS AT PRESENT,607,0.903946,0,0.0
COMMERCIAL UNITS,1,0.001489,63166,94.067014
BUILDING CLASS CATEGORY,1,0.001489,0,0.0
BUILDING CLASS AT TIME OF SALE,1,0.001489,0,0.0
LOT,1,0.001489,0,0.0


In [9]:
# Remove rows where 'GROSS SQUARE FEET' or 'LAND SQUARE FEET' are null
nyc_houses_df = nyc_houses_df.dropna(subset=['GROSS SQUARE FEET', 'LAND SQUARE FEET'])

# Verify rows were removed
print(f"New dataset shape: {nyc_houses_df.shape}")

New dataset shape: (45177, 20)


According to the glossary of variables, there are some that it makes ***NO SENSE*** to have a zero as data, therefore these will be converted to null, for a possible imputation of data.

However, this process will only be applied to columns that, by definition, do not make sense to have a value of zero (0).

This is done in this order to avoid deleting a lot of data initially because of these columns, in order to then make an imputation by the mean.

In [10]:
# Columns to nullify (replace 0 by NaN)
columns_to_nullify = [
    "GROSS SQUARE FEET",
    "LAND SQUARE FEET",
    "YEAR BUILT"
]

nyc_houses_df[columns_to_nullify] = nyc_houses_df[columns_to_nullify].replace(0, np.nan)

In [11]:
missing_and_zero_values_summary(nyc_houses_df)

,Nulls,Percentage Nulls (%),Zero values,Percentage Zeros (%)
APARTMENT NUMBER,39770,88.031520,0,0.0
GROSS SQUARE FEET,9134,20.218253,0,0.0
LAND SQUARE FEET,8276,18.319056,0,0.0
SALE PRICE,6826,15.109458,8009,17.728047
YEAR BUILT,2305,5.102154,0,0.0
TAX CLASS AT PRESENT,292,0.646347,0,0.0
BUILDING CLASS AT PRESENT,292,0.646347,0,0.0
COMMERCIAL UNITS,0,0.000000,41266,91.34294
BUILDING CLASS AT TIME OF SALE,0,0.000000,0,0.0
BUILDING CLASS CATEGORY,0,0.000000,0,0.0


The ***ADDRESS & APARTMENT NUMBER & ZIP CODE*** columns will be droped because aren't relevant for the model. In addition ***`APARTMENT NUMBER`*** has almost 90% of the data as null.

In [12]:
nyc_houses_df = nyc_houses_df.drop(columns=['ADDRESS', 'APARTMENT NUMBER', 'ZIP CODE'])

In [13]:
missing_and_zero_values_summary(nyc_houses_df)

,Nulls,Percentage Nulls (%),Zero values,Percentage Zeros (%)
GROSS SQUARE FEET,9134,20.218253,0,0.0
LAND SQUARE FEET,8276,18.319056,0,0.0
SALE PRICE,6826,15.109458,8009,17.728047
YEAR BUILT,2305,5.102154,0,0.0
BUILDING CLASS AT PRESENT,292,0.646347,0,0.0
TAX CLASS AT PRESENT,292,0.646347,0,0.0
COMMERCIAL UNITS,0,0.000000,41266,91.34294
BUILDING CLASS AT TIME OF SALE,0,0.000000,0,0.0
BUILDING CLASS CATEGORY,0,0.000000,0,0.0
SALE DATE,0,0.000000,0,0.0


The null values of ***SALE PRICE*** will be eliminated, since it makes no sense to make up values and then make a prediction based on that. In addition, it is the target variable

In [14]:
nyc_houses_df = nyc_houses_df.dropna(subset=['SALE PRICE'])
print(f"New dataset shape: {nyc_houses_df.shape}")

New dataset shape: (38351, 17)


In [15]:
missing_and_zero_values_summary(nyc_houses_df)

,Nulls,Percentage Nulls (%),Zero values,Percentage Zeros (%)
GROSS SQUARE FEET,9134,23.816850,0,0.0
LAND SQUARE FEET,8276,21.579620,0,0.0
YEAR BUILT,2302,6.002451,0,0.0
BUILDING CLASS AT PRESENT,292,0.761388,0,0.0
TAX CLASS AT PRESENT,292,0.761388,0,0.0
LOT,0,0.000000,0,0.0
COMMERCIAL UNITS,0,0.000000,35218,91.830721
BUILDING CLASS AT TIME OF SALE,0,0.000000,0,0.0
BUILDING CLASS CATEGORY,0,0.000000,0,0.0
SALE DATE,0,0.000000,0,0.0


By definition TOTAL UNITS is the sum of COMMERCIAL UNITS and RESIDENTIAL UNITS, so check that this is correct. If not, proceed with the correction.

In [17]:
# Check if TOTAL UNITS matches the sum of COMMERCIAL UNITS and RESIDENTIAL UNITS
matches = nyc_houses_df['TOTAL UNITS'].equals(nyc_houses_df['COMMERCIAL UNITS'] + nyc_houses_df['RESIDENTIAL UNITS'])

# Print result
if matches:
    print("✅ TOTAL UNITS is correctly calculated for all rows.")
else:
    print("❌ There are mismatches in TOTAL UNITS.")

# Count mismatches
mismatches = (nyc_houses_df['TOTAL UNITS'] != (nyc_houses_df['COMMERCIAL UNITS'] + nyc_houses_df['RESIDENTIAL UNITS'])).sum()
print(f"Total mismatched rows: {mismatches}")

❌ There are mismatches in TOTAL UNITS.
Total mismatched rows: 768


In [18]:
# Fix 'TOTAL UNITS' by assigning the correct sum
nyc_houses_df.loc[:, 'TOTAL UNITS'] = nyc_houses_df['COMMERCIAL UNITS'] + nyc_houses_df['RESIDENTIAL UNITS']

# Verify that all mismatches are gone
mismatches_after = (nyc_houses_df['TOTAL UNITS'] != (nyc_houses_df['COMMERCIAL UNITS'] + nyc_houses_df['RESIDENTIAL UNITS'])).sum()
print(f"✅ Total mismatched rows after correction: {mismatches_after}")

✅ Total mismatched rows after correction: 0


In [ ]:
# Remove the existing 'TOTAL UNITS' column
nyc_houses_df = nyc_houses_df.drop(columns=['TOTAL UNITS'], errors='ignore')

# Recalculate 'TOTAL UNITS' as the sum of 'COMMERCIAL UNITS' and 'RESIDENTIAL UNITS'
nyc_houses_df['TOTAL UNITS'] = nyc_houses_df['COMMERCIAL UNITS'] + nyc_houses_df['RESIDENTIAL UNITS']

# Verify the new column
nyc_houses_df[['COMMERCIAL UNITS', 'RESIDENTIAL UNITS', 'TOTAL UNITS']].sample(5)


In [20]:
missing_and_zero_values_summary(nyc_houses_df)

,Nulls,Percentage Nulls (%),Zero values,Percentage Zeros (%)
GROSS SQUARE FEET,9134,23.816850,0,0.0
LAND SQUARE FEET,8276,21.579620,0,0.0
YEAR BUILT,2302,6.002451,0,0.0
BUILDING CLASS AT PRESENT,292,0.761388,0,0.0
TAX CLASS AT PRESENT,292,0.761388,0,0.0
LOT,0,0.000000,0,0.0
COMMERCIAL UNITS,0,0.000000,35218,91.830721
BUILDING CLASS AT TIME OF SALE,0,0.000000,0,0.0
BUILDING CLASS CATEGORY,0,0.000000,0,0.0
SALE DATE,0,0.000000,0,0.0


In [44]:
nyc_houses_df.describe()

,COMMERCIAL UNITS,LOT,TOTAL UNITS,TAX CLASS AT TIME OF SALE,SALE DATE,RESIDENTIAL UNITS,GROSS SQUARE FEET,BOROUGH,BLOCK,SALE PRICE,YEAR BUILT,LAND SQUARE FEET
count,38351.0,38351.000000,38351.0,38351.000000,38351,38351.0,2.921700e+04,38351.000000,38351.000000,3.835100e+04,36049.0,3.007500e+04
mean,0.246304,235.407864,2.692759,1.472243,2017-02-25 05:50:01.298531840,2.446455,4.461029e+03,3.253840,4981.510104,1.168289e+06,1945.223806,4.112533e+03
min,0.0,1.000000,0.0,1.000000,2016-09-01 00:00:00,0.0,6.000000e+01,1.000000,1.000000,0.000000e+00,1800.0,2.000000e+00
25%,0.0,21.000000,1.0,1.000000,2016-11-29 00:00:00,1.0,1.428000e+03,3.000000,2207.000000,1.000000e+05,1920.0,2.000000e+03
50%,0.0,45.000000,1.0,1.000000,2017-02-24 00:00:00,1.0,2.000000e+03,3.000000,4664.000000,4.850000e+05,1931.0,2.500000e+03
75%,0.0,94.000000,2.0,2.000000,2017-05-25 00:00:00,2.0,2.880000e+03,4.000000,6994.500000,8.350000e+05,1964.0,3.800000e+03
max,2261.0,9009.000000,2261.0,4.000000,2017-08-31 00:00:00,894.0,1.617206e+06,5.000000,16319.000000,2.210000e+09,2017.0,3.014056e+06
std,12.033486,497.331346,18.391595,0.827560,NaN,13.831879,2.615392e+04,0.904139,3423.407280,1.470304e+07,34.372523,2.914974e+04


### ***Missing values***

In [16]:
nyc_houses_df.isna().sum()

BUILDING CLASS AT PRESENT          292
COMMERCIAL UNITS                     0
BUILDING CLASS CATEGORY              0
BUILDING CLASS AT TIME OF SALE       0
LOT                                  0
TOTAL UNITS                          0
TAX CLASS AT TIME OF SALE            0
TAX CLASS AT PRESENT               292
SALE DATE                            0
NEIGHBORHOOD                         0
RESIDENTIAL UNITS                    0
GROSS SQUARE FEET                 9134
BOROUGH                              0
BLOCK                                0
SALE PRICE                        6826
YEAR BUILT                        2305
LAND SQUARE FEET                  8276
dtype: int64

In [17]:
nulos_count = nyc_houses_features.isnull().sum()
nulos_percentage = (nulos_count / len(nyc_houses_features)) * 100

NameError: name 'nyc_houses_features' is not defined

In [ ]:
# Dataframe with the count and percentage of missing values
nulos_df = pd.DataFrame({
    'Nulos': nulos_count,
    'Porcentaje (%)': nulos_percentage
}).sort_values(by='Porcentaje (%)', ascending=False)

nulos_df

,Nulos,Porcentaje (%)
GROSS SQUARE FEET,21897,32.671362
LAND SQUARE FEET,20799,31.033094
SALE PRICE,11354,16.940706
BUILDING CLASS AT PRESENT,607,0.905673
TAX CLASS AT PRESENT,607,0.905673
COMMERCIAL UNITS,1,0.001492
TOTAL UNITS,1,0.001492
BUILDING CLASS CATEGORY,1,0.001492
LOT,1,0.001492
BUILDING CLASS AT TIME OF SALE,1,0.001492


***The number of nulls less than 30%, therefore they will be imputed by mode.***

In [ ]:
duplicate_rows = nyc_houses_features.duplicated().sum()
print("Number of duplicate rows: ", duplicate_rows)

Number of duplicate rows:  70254


In [ ]:
nyc_houses_features = nyc_houses_features.drop_duplicates()
nyc_houses_features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 67022 entries, 0 to 67637
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   BUILDING CLASS AT PRESENT       66415 non-null  category      
 1   COMMERCIAL UNITS                67021 non-null  Int64         
 2   BUILDING CLASS CATEGORY         67021 non-null  category      
 3   BUILDING CLASS AT TIME OF SALE  67021 non-null  category      
 4   LOT                             67021 non-null  float64       
 5   TOTAL UNITS                     67021 non-null  Int64         
 6   TAX CLASS AT TIME OF SALE       67021 non-null  float64       
 7   TAX CLASS AT PRESENT            66415 non-null  category      
 8   ZIP CODE                        67021 non-null  float64       
 9   SALE DATE                       67021 non-null  datetime64[ns]
 10  NEIGHBORHOOD                    67021 non-null  category      
 11  RESIDEN

In [ ]:
cols_categorical = ['TAX CLASS AT TIME OF SALE', 'LOT', 'ZIP CODE', 'BOROUGH', 'BLOCK']
nyc_houses_features[cols_categorical] = nyc_houses_features[cols_categorical].astype('category')

In [ ]:
nyc_houses_features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 67022 entries, 0 to 67637
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   BUILDING CLASS AT PRESENT       66415 non-null  category      
 1   COMMERCIAL UNITS                67021 non-null  Int64         
 2   BUILDING CLASS CATEGORY         67021 non-null  category      
 3   BUILDING CLASS AT TIME OF SALE  67021 non-null  category      
 4   LOT                             67021 non-null  category      
 5   TOTAL UNITS                     67021 non-null  Int64         
 6   TAX CLASS AT TIME OF SALE       67021 non-null  category      
 7   TAX CLASS AT PRESENT            66415 non-null  category      
 8   ZIP CODE                        67021 non-null  category      
 9   SALE DATE                       67021 non-null  datetime64[ns]
 10  NEIGHBORHOOD                    67021 non-null  category      
 11  RESIDEN

### ***PipeLines***

In [ ]:
cols_numeric = ['COMMERCIAL UNITS', 'TOTAL UNITS', 'RESIDENTIAL UNITS', 'GROSS SQUARE FEET', 'SALE PRICE', 'YEAR BUILT', 'LAND SQUARE FEET']
cols_categoric = ['BUILDING CLASS AT PRESENT', 'BUILDING CLASS CATEGORY', 'BUILDING CLASS AT TIME OF SALE', 'LOT', 'ZIP CODE', 'NEIGHBORHOOD', 'BOROUGH', 'BLOCK']
cols_categoric_ord = ['TAX CLASS AT TIME OF SALE', 'TAX CLASS AT PRESENT']

In [ ]:
numeric_pipe = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
    ]
)

categorical_pipe = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder()),
    ]
)

categorical_ord_pipe = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OrdinalEncoder()),
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("numeric", numeric_pipe, cols_numeric),
        ("categoric", categorical_pipe, cols_categoric),
        ("categoric ordinales", categorical_ord_pipe, cols_categoric_ord),
    ]
)


In [ ]:
preprocessor

ColumnTransformer(transformers=[('numeric',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 ['COMMERCIAL UNITS', 'TOTAL UNITS',
                                  'RESIDENTIAL UNITS', 'GROSS SQUARE FEET',
                                  'SALE PRICE', 'YEAR BUILT',
                                  'LAND SQUARE FEET']),
                                ('categoric',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot', OneHotEncoder())]),
                                 ['BUILDING CLASS AT PRESENT',
                                  'BUILDING CLASS CATEGORY',
                                  'BUILDING CLASS AT TIME OF SALE', 'LOT',
                                  'ZIP CODE', 'NEIGHBORHOOD', 'BOROUGH',
                                  'BLOCK']),
                                ('categoric ordinales',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot', OrdinalEncoder())]),
                                 ['TAX CLASS AT TIME OF SALE',
                                  'TAX CLASS AT PRESENT'])])

#### Example of the data preprocessing pipeline

***Train / Test split***

In [ ]:
X_features = nyc_houses_features.drop("SALE PRICE", axis="columns")
Y_target = nyc_houses_features["SALE PRICE"]

# 80% train, 20% test
x_train, x_test, y_train, y_test = train_test_split(
    X_features, Y_target, test_size=0.2, stratify=Y_target
)


ValueError: Input y contains NaN.